In [1]:
! pip freeze > requirements.txt

In [1]:
from PIL import Image
import albumentations as A
import numpy as np
import os
import random

In [2]:
# apply transform to the input image and return newly generated image
def transform_image(file_name, transforms):
    img = np.array(Image.open(file_name).convert('RGB'))
    transformed = transforms(image=img)
    transformed_image = transformed['image']
    return transformed_image

In [4]:
T = A.Compose([
    A.Resize(256,256),
    A.RandomCrop(224,224),
    # randomly apply one of following three transform
    A.OneOf([
        A.VerticalFlip(p=1),
        A.HorizontalFlip(p=1),
        A.RandomRotate90(p=1)
    ]),
    # randomly apply one of following three transform
    A.OneOf([
        A.ColorJitter(p=1),
        A.Equalize(p=1),
        A.GaussNoise(p=1),
        A.RandomBrightnessContrast(p=1)
    ])   
])

In [5]:
image_base_dir = "../datasets/train_set_augmented/train/"
class_names = os.listdir(image_base_dir)

# apply transform to all image in 'image_base_dir'
for name in class_names:
    # read list of folders
    sub_dir = image_base_dir+name+"/"
    files = os.listdir(sub_dir)
    # read each folder
    for file in files:
        new_img = transform_image(sub_dir+file, transforms=T)
        try:
            new_img = Image.fromarray(new_img, "RGB")
            new_img.save(sub_dir+"augmented_"+file,'JPEG')
        except:
            continue
        